## Buy AAPL

To buy a stock, you send information about what you want to buy to IB. The `app.placeOrder` method does all the hard work for you.

First, import the classes from the IB API. Import threading to run the app in a single thread and time to pause execution of the script.

In [15]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

This class creates the app. The app is what you use to interact with IB. This class inherits two classes from the IB API. If this doesn’t make sense, don’t worry. The API needs an order ID associated with every order. `nextValidId` is a built in function that finds the next available order ID.

In [16]:
class Parent(object):
    name = "Roger"

In [17]:
class Child(Parent):
    def father_name(self):
        return self.name

In [18]:
jason = Child()

In [19]:
print(jason.father_name())

Roger


In [20]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print("The next valid order id is: ", self.nextorderId)

    def orderStatus(
        self,
        orderId,
        status,
        filled,
        remaining,
        avgFullPrice,
        permId,
        parentId,
        lastFillPrice,
        clientId,
        whyHeld,
        mktCapPrice,
    ):
        print(
            "orderStatus - orderid:",
            orderId,
            "status:",
            status,
            "filled",
            filled,
            "remaining",
            remaining,
            "lastFillPrice",
            lastFillPrice,
        )

    def openOrder(self, orderId, contract, order, orderState):
        print(
            "openOrder id:",
            orderId,
            contract.symbol,
            contract.secType,
            "@",
            contract.exchange,
            ":",
            order.action,
            order.orderType,
            order.totalQuantity,
            orderState.status,
        )

    def execDetails(self, reqId, contract, execution):
        print(
            "Order Executed: ",
            reqId,
            contract.symbol,
            contract.secType,
            contract.currency,
            execution.execId,
            execution.orderId,
            execution.shares,
            execution.lastLiquidity,
        )

Next, create a function to run the app. Then, define a stock contract (IB calls everything a contract).

In [21]:
def run_loop():
    app.run()


def stock_contract(
    symbol,
    secType='STK',
    exchange='SMART',
    currency='USD'
):
    # create a stock contract
    contract = Contract()
    
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency

    return contract

Now that the setup is out of the way, make the connection and start a thread. The while loop checks if the API is connected. If it is, `app.nextorderId` returns an `int`. Otherwise it returns `None`.

In [25]:
app = IBapi()
app.connect('127.0.0.1', 7497, 123)

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
    if isinstance(app.nextorderId, int):
        print('connected')
        break
    else:
        print('waiting for connection')
        time.sleep(1)

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


waiting for connection
The next valid order id is:  1
connected


This is where the core logic of your algorithm lives. In this case, I keep it simple and just 10 shares of AAPL.

Create an order and send it to IB. IB expects the order to be a Python object with buy or sell, the quantity, and order type. Because I send a limit order, I set the limit price.

Finally, I send the order, wait three seconds, cancel it for the sake of this demo, wait three more seconds, then disconnect the app.

In [27]:
order = Order()

order.action = "BUY"
order.totalQuantity = 100
order.orderType = "LMT"
order.lmtPrice = "175.00"
order.eTradeOnly = ""
order.firmQuoteOnly = ""

app.nextorderId += 1

app.placeOrder(
    app.nextorderId, 
    stock_contract("AAPL"), 
    order
)

openOrder id: 3 AAPL STK @ SMART : BUY LMT 100 Submitted
orderStatus - orderid: 3 status: Submitted filled 0 remaining 100 lastFillPrice 0.0


In [28]:
# cancel the order for the demo
print('cancelling order')
app.cancelOrder(app.nextorderId, "")

time.sleep(15)
app.disconnect()

ERROR 3 202 Order Canceled - reason:


cancelling order
orderStatus - orderid: 3 status: Cancelled filled 0 remaining 100 lastFillPrice 0.0
